# Recherche d'information : librairie PyTerrier

Dans cette partie, nous nous intéressons à la librairie [PyTerrier](https://pyterrier.readthedocs.io/en/latest/#) qui permet de mettre en place diverses briques d'un moteur de recherche.
PyTerrier est basée sur [Terrier](http://terrier.org/) qui est un moteur de recherche développé en Java.

Nous allons voir :
*   l'installation et la configuration
*   l'indexation d'une collection
*   l'accès à l'index
*   l'évaluation d'un moteur de recherche


## Installation ete configuration

Après l'installation de la librairie, il est nécessaire d'initialiser PyTerrier pour importer les fichiers jar et démarrer la machine virtuelle associée;

In [ ]:
!pip install python-terrier

import pyterrier as pt
if not pt.started():
  pt.init(boot_packages=["com.github.terrierteam:terrier-prf:-SNAPSHOT"])

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 107.6/107.6 kB 2.7 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.5/1.5 MB 12.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 69.6/69.6 kB 8.8 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 335.2/335.2 kB 16.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 48.8/48.8 kB 5.0 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 116.3/116.3 kB 12.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 287.4/287.4 kB 16.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 45.1/45.1 kB 4.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 45.4/45.4 kB 4.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.3/1.3 MB 20.2 MB/s eta 0:00:00
  Preparing metadata (setup

PyTerrier 0.10.0 has loaded Terrier 5.8 (built by craigm on 2023-11-01 18:05) and terrier-helper 0.0.8



## Indexation d'une collection

Il est possible d'indexer plusieurs formats de collection : format TREC, fichiers en texte brut ou en PDF, ou encore des Dataframe Pandas ([pour plus de détails](https://pyterrier.readthedocs.io/en/latest/terrier-indexing.html)).

Un petit exemple à titre illustratif est foourni dans le code suivant :

In [ ]:
import pandas as pd

# configuration de l'affichage
pd.set_option('display.max_colwidth', 150)

# le jeu de données au format DataFrame
docs_df = pd.DataFrame([
        ["d1", "this is the first document of many documents"],
        ["d2", "this is another document"],
        ["d3", "the topic of this document is unknown"]
    ], columns=["docno", "text"])

# création de l'index
indexer = pt.DFIndexer("./index_3docs", overwrite=True)         # Définition du format de données (DFIndexer())
index_ref = indexer.index(docs_df["text"], docs_df["docno"])    # Indexation des champs text et docno
!ls -lh index_3docs/                                            # Affichage de l'index sauvegardé dans "./index_3docs/"

total 40K
-rw-r--r-- 1 root root    3 Mar 20 08:25 data.direct.bf
-rw-r--r-- 1 root root   51 Mar 20 08:25 data.document.fsarrayfile
-rw-r--r-- 1 root root    4 Mar 20 08:25 data.inverted.bf
-rw-r--r-- 1 root root  344 Mar 20 08:25 data.lexicon.fsomapfile
-rw-r--r-- 1 root root  249 Mar 20 08:25 data.lexicon.fsomaphash
-rw-r--r-- 1 root root   33 Mar 20 08:25 data.meta-0.fsomapfile
-rw-r--r-- 1 root root   24 Mar 20 08:25 data.meta.idx
-rw-r--r-- 1 root root   48 Mar 20 08:25 data.meta.zdata
-rw-r--r-- 1 root root 4.1K Mar 20 08:25 data.properties


De nombreux fichiers sont créés : index direct, index inverse, méta-données de l'index et de la configuration de l'indexation, etc...


Il est également possible de modifier la configuration de l'indexation : [voir ici](https://pyterrier.readthedocs.io/en/latest/terrier-indexing.html#indexing-configuration).

Pour chager un index existant en local :

In [ ]:
index = pt.IndexFactory.of(index_ref)

Il est aussi possible de voir les statistiques de l'index.
pour connaître toutes les fonctions d'interrogation, se référencer à la [javadoc](http://terrier.org/docs/current/javadoc/org/terrier/structures/Index.html).

In [ ]:
# statistiques de la collection
print(index.getCollectionStatistics().toString())

Number of documents: 3
Number of terms: 4
Number of postings: 6
Number of fields: 0
Number of tokens: 7
Field names: []
Positions:   false



In [ ]:
# statistiques du vocabulaire.
# Nt : document frequency : nombre de documents contenant le terme
# TF : term frequency : nombre d'occurences du terme
# maxTF : nombre d'occurences maximal pour un document
for kv in index.getLexicon():
  print("%s (%s) -> %s (%s)" % (kv.getKey(), type(kv.getKey()), kv.getValue().toString(), type(kv.getValue()) ) )

document (<class 'str'>) -> term0 Nt=3 TF=4 maxTF=2 @{0 0 0} (<class 'jnius.reflect.org.terrier.structures.LexiconEntry'>)
first (<class 'str'>) -> term1 Nt=1 TF=1 maxTF=1 @{0 0 7} (<class 'jnius.reflect.org.terrier.structures.LexiconEntry'>)
topic (<class 'str'>) -> term2 Nt=1 TF=1 maxTF=1 @{0 1 1} (<class 'jnius.reflect.org.terrier.structures.LexiconEntry'>)
unknown (<class 'str'>) -> term3 Nt=1 TF=1 maxTF=1 @{0 1 5} (<class 'jnius.reflect.org.terrier.structures.LexiconEntry'>)


In [ ]:
# focus sur un terme particulier
index.getLexicon()["document"].toString()

'term0 Nt=3 TF=4 maxTF=2 @{0 0 0}'

In [ ]:
# récupère les statistiques de l'index inverse à partir d'un terme particulier
pointer = index.getLexicon()["document"]
for posting in index.getInvertedIndex().getPostings(pointer):
    print(posting.toString() + " doclen=%d" % posting.getDocumentLength())

ID(0) TF(2) doclen=3
ID(1) TF(1) doclen=1
ID(2) TF(1) doclen=3



De plus, PyTerrier met à disposition [une collection de jeux de données pré-traités](https://pyterrier.readthedocs.io/en/latest/datasets.html).
Dans ce qui suit, nous allons nous concentrer sur le jeu de données CORD19 qui recense des articles liés à la crise sanitaire Covid-19. Il est

In [ ]:
dataset = pt.datasets.get_dataset('irds:cord19/trec-covid')
indexer = pt.index.IterDictIndexer('./cord19-index')
indexref = indexer.index(dataset.get_corpus_iter(), fields=('title', 'abstract'))
index = pt.IndexFactory.of(indexref)

[INFO] [starting] building docstore
[INFO] If you have a local copy of https://ai2-semanticscholar-cord-19.s3-us-west-2.amazonaws.com/2020-07-16/metadata.csv, you can symlink it here to avoid downloading it again: /root/.ir_datasets/downloads/80d664e496b8b7e50a39c6f6bb92e0ef
[INFO] [starting] https://ai2-semanticscholar-cord-19.s3-us-west-2.amazonaws.com/2020-07-16/metadata.csv
docs_iter:   0%|                                    | 0/192509 [00:00<?, ?doc/s]
https://ai2-semanticscholar-cord-19.s3-us-west-2.amazonaws.com/2020-07-16/metadata.csv: 0.0%| 0.00/269M [00:00<?, ?B/s]
https://ai2-semanticscholar-cord-19.s3-us-west-2.amazonaws.com/2020-07-16/metadata.csv: 0.0%| 24.6k/269M [00:00<26:19, 170kB/s]
https://ai2-semanticscholar-cord-19.s3-us-west-2.amazonaws.com/2020-07-16/metadata.csv: 0.0%| 57.3k/269M [00:00<22:24, 200kB/s]
https://ai2-semanticscholar-cord-19.s3-us-west-2.amazonaws.com/2020-07-16/metadata.csv: 0.0%| 90.1k/269M [00:00<21:18, 210kB/s]
https://ai2-semanticscholar-cord-1

cord19/trec-covid documents:   0%|          | 0/192509 [00:00<?, ?it/s]

08:38:06.294 [ForkJoinPool-1-worker-3] WARN org.terrier.structures.indexing.Indexer - Adding an empty document to the index (8is9x9sc) - further warnings are suppressed
08:39:39.430 [ForkJoinPool-1-worker-3] ERROR org.terrier.structures.indexing.Indexer - Could not finish MetaIndexBuilder: 
java.io.IOException: Key 8lqzfj2e is not unique: 37597,11755
For MetaIndex, to suppress, set metaindex.compressed.reverse.allow.duplicates=true
	at org.terrier.structures.collections.FSOrderedMapFile$MultiFSOMapWriter.mergeTwo(FSOrderedMapFile.java:1374)
	at org.terrier.structures.collections.FSOrderedMapFile$MultiFSOMapWriter.close(FSOrderedMapFile.java:1308)
	at org.terrier.structures.indexing.BaseMetaIndexBuilder.close(BaseMetaIndexBuilder.java:321)
	at org.terrier.structures.indexing.classical.BasicIndexer.indexDocuments(BasicIndexer.java:270)
	at org.terrier.structures.indexing.classical.BasicIndexer.createDirectIndex(BasicIndexer.java:388)
	at org.terrier.structures.indexing.Indexer.index(Inde

**Exercice 1**

Afficher les statistiques de l'index Cord19 et analyser statistiques du terme "tv" (pas trop fréquent pour question d'affichage).

In [ ]:
collec = index.getCollectionStatistics().toString()
print(collec)

pointer = index.getLexicon()["tv"]
print(pointer)

for posting in index.getInvertedIndex().getPostings(pointer):
    print(posting.toString() + " doclen=%d" % posting.getDocumentLength())

# (2777,3,F[0,3]) doclen=230 -> id document, frequence documentaire du terme (pour ce doc), pointeurs vers d'autres documents, taille du doc

Number of documents: 192509
Number of terms: 158515
Number of postings: 12290426
Number of fields: 2
Number of tokens: 19603234
Field names: [title, abstract]
Positions:   false

term18986 Nt=70 TF=183 maxTF=2147483647 @{0 21226724 1} TFf=3,180
(2777,3,F[0,3]) doclen=230
(10054,1,F[0,1]) doclen=273
(11115,3,F[0,3]) doclen=239
(11396,1,F[0,1]) doclen=187
(11998,1,F[0,1]) doclen=75
(15443,2,F[0,2]) doclen=73
(22159,1,F[0,1]) doclen=80
(24922,2,F[1,1]) doclen=60
(29671,2,F[0,2]) doclen=174
(32156,1,F[0,1]) doclen=272
(38532,8,F[1,7]) doclen=181
(42346,5,F[0,5]) doclen=250
(43113,10,F[0,10]) doclen=206
(43829,1,F[0,1]) doclen=214
(49420,2,F[0,2]) doclen=164
(50340,4,F[0,4]) doclen=205
(51298,4,F[0,4]) doclen=161
(51965,1,F[0,1]) doclen=73
(55321,2,F[0,2]) doclen=262
(56159,1,F[0,1]) doclen=240
(56638,4,F[0,4]) doclen=157
(60575,1,F[0,1]) doclen=144
(61226,4,F[0,4]) doclen=253
(61831,1,F[0,1]) doclen=201
(62509,1,F[0,1]) doclen=238
(64784,6,F[0,6]) doclen=166
(69551,5,F[0,5]) doclen=109
(70

Le résultat attendu débute ainsi :


```
Number of documents: 192509
Number of terms: 158515
Number of postings: 12290426
Number of fields: 2
Number of tokens: 19603234
Field names: [title, abstract]
Positions:   false

term18986 Nt=70 TF=183 maxTF=2147483647 @{0 21226724 1} TFf=3,180
(2777,3,F[0,3]) doclen=230
(10054,1,F[0,1]) doclen=273
(11115,3,F[0,3]) doclen=239
(11396,1,F[0,1]) doclen=187
(11998,1,F[0,1]) doclen=75
(15443,2,F[0,2]) doclen=73
(22159,1,F[0,1]) doclen=80
(24922,2,F[1,1]) doclen=60
(29671,2,F[0,2]) doclen=174
(32156,1,F[0,1]) doclen=272
(38532,8,F[1,7]) doclen=181
(42346,5,F[0,5]) doclen=250
(43113,10,F[0,10]) doclen=206
(43829,1,F[0,1]) doclen=214
(49420,2,F[0,2]) doclen=164
(50340,4,F[0,4]) doclen=205
```




## Recherche de documents à partir de l'index

Pour effectuer une recherche dans l'index, il faut utiliser la fonction batchRetrieve qui prend en paramètre l'index et le modèle de pondération (tf, tf-idf, etc...). La liste des modèles supportés est disponible [ici](http://terrier.org/docs/current/javadoc/org/terrier/matching/models/package-summary.html).

In [ ]:
br = pt.BatchRetrieve(index, wmodel="Tf")
br.search("chemical reactions")

,qid,docid,docno,rank,score,query
0,1,12409,ij3ncdb6,0,21.0,chemical reactions
1,1,24432,094bk0t0,1,14.0,chemical reactions
2,1,11268,qroxmo85,2,13.0,chemical reactions
3,1,120041,pb00yr0r,3,13.0,chemical reactions
4,1,177724,44jyy79k,4,13.0,chemical reactions
...,...,...,...,...,...,...
995,1,65110,1kg5rild,995,2.0,chemical reactions
996,1,65338,oxt0ly5b,996,2.0,chemical reactions
997,1,65511,vkzwn3t8,997,2.0,chemical reactions
998,1,65585,41fsmwzj,998,2.0,chemical reactions


On récupère alors un DataFrame dont les colonnes sont les suivantes :
*   qid : identifiant de la requête. Par défaut, il s'agit de "1", puisqu'il s'agit de notre première et unique requête.
*   docid : l'identifiant interne de Terrier pour chaque document
*   docno : l'identifiant unique externe (chaîne de caractères) pour chaque document
*   score : score des documents selon le modèle choisi (ici : fréquence totale des tf des termes de la requête dans chaque document)
*   rank : rang du document dénotant l'ordre décroissant par score.
*   query : la requête d'entrée

In [ ]:
# autre exemple de modèle : TF-IDF
tfidf = pt.BatchRetrieve(index, wmodel="TF_IDF")
tfidf.search("chemical reactions")

,qid,docid,docno,rank,score,query
0,1,18717,iavwkdpr,0,11.379339,chemical reactions
1,1,121217,msdycum2,1,10.602074,chemical reactions
2,1,171636,v3blnh02,2,10.364436,chemical reactions
3,1,147193,ei4rb8fr,3,10.352866,chemical reactions
4,1,170863,sj8i9ss2,4,9.433666,chemical reactions
...,...,...,...,...,...,...
995,1,6381,wvw7alsc,995,4.039239,chemical reactions
996,1,32060,7z2pfqfb,996,4.039239,chemical reactions
997,1,91880,8lu58q4k,997,4.039239,chemical reactions
998,1,135920,xvzhb7ix,998,4.039239,chemical reactions


On peut aussi fournir plusieurs requêtes grâce à un dataFrame. Pour interroger l'index, on applique la fonction transform() au BatchRetriever (br).
pour plus de détails, voir [les propriétés des transformations](https://pyterrier.readthedocs.io/en/latest/transformer.html) ainsi que les [opérations possibles](https://pyterrier.readthedocs.io/en/latest/operators.html).

In [ ]:
import pandas as pd
queries = pd.DataFrame([["q1", "document"], ["q2", "first document"]], columns=["qid", "query"])
br.transform(queries)       # ou aussi : br(queries)


,qid,docid,docno,rank,score,query
0,q1,14569,mm7gxjf1,0,13.0,document
1,q1,137615,duxm9u8v,1,12.0,document
2,q1,189505,hxp258y8,2,11.0,document
3,q1,18789,x3o3a45b,3,10.0,document
4,q1,37445,q5wglpoj,4,10.0,document
...,...,...,...,...,...,...
1995,q2,89264,2kscr64t,995,3.0,first document
1996,q2,89265,0r1kojbq,996,3.0,first document
1997,q2,89737,icxug96f,997,3.0,first document
1998,q2,89738,kkgpfpjv,998,3.0,first document


**Exercice 2**

Ordonnancer les documents pour 3 requêtes : "covid disease", "hospital" et "home".
La fonction d'ordonnacement devra être de la forme suivante :


```
0.4 * score_Bm25 + 0.6 * score_Dirichlet
```


In [ ]:
queries = pd.DataFrame([["q1", "covid disease"], ["q2", "hospital"], ["q3", "home"]], columns=["qid", "query"])

br_DPH = pt.BatchRetrieve(index, wmodel="DirichletLM")
br_BM25 = pt.BatchRetrieve(index, wmodel="BM25")
res = (0.4 * br_BM25 + 0.6 * br_DPH).transform(queries)
res



,qid,docid,docno,score,query,rank
0,q1,43498.0,9d8go2sl,2.089712,covid disease,5
1,q1,106017.0,a0w3n30z,2.089712,covid disease,6
2,q1,106018.0,smyp9fyo,2.089712,covid disease,7
3,q1,106019.0,vtfn9y5p,2.089712,covid disease,8
4,q1,41482.0,w1vsm3rq,1.873016,covid disease,84
...,...,...,...,...,...,...
4438,q3,NaN,3kp3ptbn,1.018824,NaN,1433
4439,q3,NaN,7j98rhr8,1.018824,NaN,1434
4440,q3,NaN,j1gi1cxu,1.018506,NaN,1435
4441,q3,NaN,uknmoxu3,1.018188,NaN,1436


Résultat attendu :    


```
     qid     docid     docno     score          query  rank
0     q1   43498.0  9d8go2sl  2.089712  covid disease     5
1     q1  106017.0  a0w3n30z  2.089712  covid disease     6
2     q1  106018.0  smyp9fyo  2.089712  covid disease     7
3     q1  106019.0  vtfn9y5p  2.089712  covid disease     8
4     q1   41482.0  w1vsm3rq  1.873016  covid disease    84
...   ..       ...       ...       ...            ...   ...
4727  q3       NaN  3kp3ptbn  1.018824            NaN  1433
4728  q3       NaN  7j98rhr8  1.018824            NaN  1434
4729  q3       NaN  j1gi1cxu  1.018506            NaN  1435
4730  q3       NaN  uknmoxu3  1.018188            NaN  1436
4731  q3       NaN  oe2496ql  1.017870            NaN  1437

[4732 rows x 6 columns]
     qid  docid_x     docno  rank_x   score_x        query_x  docid_y  rank_y  \
0     q1    43498  9d8go2sl       0  3.728774  covid disease    43498      33   
1     q1   106017  a0w3n30z       1  3.728774  covid disease   106017      34   
2     q1   106018  smyp9fyo       2  3.728774  covid disease   106018      35   
3     q1   106019  vtfn9y5p       3  3.728774  covid disease   106019      36   
4     q1    41482  w1vsm3rq       4  3.666853  covid disease    41482     509   
...   ..      ...       ...     ...       ...            ...      ...     ...   
1312  q3    92583  of5dvq44     995  7.577804           home    92583     661   
1313  q3    92584  hgvt49x1     996  7.577804           home    92584     662   
1314  q3   180665  mob8dbcr     997  7.577804           home   180665     663   
1315  q3   172568  g73syut2     998  7.577420           home   172568     432   
1316  q3   139621  p2ro0zdj     999  7.557501           home   139621     664   

       score_y        query_y     score  
0     0.997005  covid disease  2.089712  
1     0.997005  covid disease  2.089712  
2     0.997005  covid disease  2.089712  
3     0.997005  covid disease  2.089712  
4     0.677124  covid disease  1.873016  
...        ...            ...       ...  
1312  1.757552           home  4.085653  
1313  1.757552           home  4.085653  
1314  1.757552           home  4.085653  
1315  2.159763           home  4.326826  
1316  1.756999           home  4.077200  

[1317 rows x 11 columns]
```



**Excercice 3**

Créez un ordonnanceur qui effectue les opérations suivantes :
* obtient les 10 documents les mieux notés par fréquence de terme (wmodel="Tf")
* obtenir les 10 documents les mieux notés par TF.IDF (wmodel="TF_IDF")
* ré-ordonne uniquement les documents trouvés dans les DEUX paramètres de recherche précédents en utilisant BM25.

Combien de documents sont récupérés par ce pipeline complet pour la requête "chemical"?

Vérification : le document avec le docno "37771" devrait avoir un score de 12.426309 $ pour la requête "chemical".

In [ ]:
query = 'chemical'

br_tfidf = pt.BatchRetrieve(index, wmodel="TF_IDF")
br_tf = pt.BatchRetrieve(index, wmodel="Tf")
br_bm25 = pt.BatchRetrieve(index, wmodel='BM25')


res_inter = ((br_tf%10 & br_tfidf%10) >> br_bm25).search(query)
res_inter



,qid,docid,docno,rank,score,query
0,1,37771,jn5qi1jb,0,12.607699,chemical
1,1,134305,0smev8vt,1,12.426464,chemical
2,1,56631,sps45fj5,2,11.883608,chemical


Résultat attendu :    


```
  qid   docid     docno  rank      score     query
0  q1   37771  jn5qi1jb     0  12.607699  chemical
1  q1  134305  0smev8vt     1  12.426464  chemical
2  q1   56631  sps45fj5     2  11.883608  chemical
```



## Reformulation de requêtes

Il est également possible de mettre en place des pipelines de [reformulation de requêtes](https://pyterrier.readthedocs.io/en/latest/rewrite.html).

In [ ]:
bo1 = pt.rewrite.Bo1QueryExpansion(index)
dph = pt.BatchRetrieve(index, wmodel="DPH")
pipeline = dph >> bo1 >> dph
pipeline.search("chemical reactions")

,qid,docid,docno,rank,score,query_0,query
0,1,147193,ei4rb8fr,0,22.239598,chemical reactions,applypipeline:off chemic^1.449575286 reaction^1.300223703 hazard^0.339724370 reactiv^0.307948549 bioprocess^0.301225621 combust^0.277576853 explos...
1,1,171636,v3blnh02,1,22.204758,chemical reactions,applypipeline:off chemic^1.449575286 reaction^1.300223703 hazard^0.339724370 reactiv^0.307948549 bioprocess^0.301225621 combust^0.277576853 explos...
2,1,18717,iavwkdpr,2,11.638978,chemical reactions,applypipeline:off chemic^1.449575286 reaction^1.300223703 hazard^0.339724370 reactiv^0.307948549 bioprocess^0.301225621 combust^0.277576853 explos...
3,1,20409,1g9kmpdi,3,10.942838,chemical reactions,applypipeline:off chemic^1.449575286 reaction^1.300223703 hazard^0.339724370 reactiv^0.307948549 bioprocess^0.301225621 combust^0.277576853 explos...
4,1,121217,msdycum2,4,10.616867,chemical reactions,applypipeline:off chemic^1.449575286 reaction^1.300223703 hazard^0.339724370 reactiv^0.307948549 bioprocess^0.301225621 combust^0.277576853 explos...
...,...,...,...,...,...,...,...
995,1,2550,lar2m59n,995,4.090489,chemical reactions,applypipeline:off chemic^1.449575286 reaction^1.300223703 hazard^0.339724370 reactiv^0.307948549 bioprocess^0.301225621 combust^0.277576853 explos...
996,1,29950,lpckymwx,996,4.082916,chemical reactions,applypipeline:off chemic^1.449575286 reaction^1.300223703 hazard^0.339724370 reactiv^0.307948549 bioprocess^0.301225621 combust^0.277576853 explos...
997,1,2556,l7s3raop,997,4.081913,chemical reactions,applypipeline:off chemic^1.449575286 reaction^1.300223703 hazard^0.339724370 reactiv^0.307948549 bioprocess^0.301225621 combust^0.277576853 explos...
998,1,135976,1397kds0,998,4.081913,chemical reactions,applypipeline:off chemic^1.449575286 reaction^1.300223703 hazard^0.339724370 reactiv^0.307948549 bioprocess^0.301225621 combust^0.277576853 explos...


L'autre solution est de l'intégrer directement dans la fonction d'ordonnancement. Mais la requête reformulée n'est pas visible et la solution précédente fait prendre conscience de la pipeline faite par le système de RI (ranking >> reformulation >> ranking quand on utilise des modèles basés sur la relevance feedback. ou reformulation >> ranking sinon). Plus d'exemples [ici](https://pyterrier.readthedocs.io/en/latest/rewrite.html).

In [ ]:
# modèle DPH avant reformulation de requête
pipelineQE = pt.BatchRetrieve(index, wmodel="DPH", controls={"qemodel" : "Bo1", "qe" : "off"})
pipelineQE.search("chemical reactions")

,qid,docid,docno,rank,score,query
0,1,18717,iavwkdpr,0,12.188020,chemical reactions
1,1,171636,v3blnh02,1,11.867153,chemical reactions
2,1,147193,ei4rb8fr,2,11.858933,chemical reactions
3,1,121217,msdycum2,3,11.136464,chemical reactions
4,1,170863,sj8i9ss2,4,10.046963,chemical reactions
...,...,...,...,...,...,...
995,1,6298,d752zg2b,995,4.256810,chemical reactions
996,1,106503,f3y5m6yg,996,4.254522,chemical reactions
997,1,140007,8s1lw59s,997,4.254522,chemical reactions
998,1,79248,iazolbhs,998,4.252125,chemical reactions


In [ ]:
# modèle DPH après reformulation de requête
pipelineQE = pt.BatchRetrieve(index, wmodel="DPH", controls={"qemodel" : "Bo1", "qe" : "on"})
pipelineQE.search("chemical reactions")

,qid,docid,docno,rank,score,query
0,1,147193,ei4rb8fr,0,22.239598,chemical reactions
1,1,171636,v3blnh02,1,22.204758,chemical reactions
2,1,18717,iavwkdpr,2,11.638978,chemical reactions
3,1,20409,1g9kmpdi,3,10.942838,chemical reactions
4,1,121217,msdycum2,4,10.616867,chemical reactions
...,...,...,...,...,...,...
995,1,2550,lar2m59n,995,4.090489,chemical reactions
996,1,29950,lpckymwx,996,4.082916,chemical reactions
997,1,2556,l7s3raop,997,4.081913,chemical reactions
998,1,135976,1397kds0,998,4.081913,chemical reactions


### Evaluation d'un système de recherche d'information

Pour évaluer un système de RI, il est nécessaire d'avoir un jeu de données constitué de requêtes et de jugements de pertinence.

In [ ]:
# exemple de 5 requêtes pour cord19
dataset.get_topics(variant='title').head(5)

[INFO] [starting] https://ir.nist.gov/covidSubmit/data/topics-rnd5.xml
[INFO] [finished] https://ir.nist.gov/covidSubmit/data/topics-rnd5.xml: [00:00] [18.7kB] [25.1MB/s]


,qid,query
0,1,coronavirus origin
1,2,coronavirus response to weather changes
2,3,coronavirus immunity
3,4,how do people die from the coronavirus
4,5,animal models of covid 19


In [ ]:
# exemple de jugements de pertinence pour les 5 premières requêtes
dataset.get_qrels().head(5)

[INFO] [starting] https://ir.nist.gov/covidSubmit/data/qrels-covid_d5_j0.5-5.txt
[INFO] [finished] https://ir.nist.gov/covidSubmit/data/qrels-covid_d5_j0.5-5.txt: [00:00] [1.14MB] [8.83MB/s]


,qid,docno,label,iteration
0,1,005b2j4b,2,4.5
1,1,00fmeepz,1,4
2,1,010vptx3,2,0.5
3,1,0194oljo,1,2.5
4,1,021q9884,1,4


**Exercice 4**

A partir des requêtes et des jugements de pertinence du jeu de données CORD19, Ecrire le code qui permet d'afficher les résultats de la première requête de Cord19. L'affichage fusionnera les colonnes retournées par le BatchRetriever et les colonnes des qrels (merge sur qid et docno pour rajouter label et iteration au tableau).

In [ ]:
br_bm25 = pt.BatchRetrieve(index, wmodel='TF_IDF')
topics = dataset.get_topics(variant='title').head(1)
res = br_bm25.transform(topics)
qrels = dataset.get_qrels()
merged_res = pd.merge(res, qrels, on=['qid', 'docno'])
merged_res

,qid,docid,docno,rank,score,query,label,iteration
0,1,122804,75773gwg,0,6.948973,coronavirus origin,2,5
1,1,122805,kn2z7lho,1,6.948973,coronavirus origin,2,3
2,1,122806,4fb291hq,2,6.948973,coronavirus origin,1,3
3,1,135326,ne5r4d4b,3,6.948973,coronavirus origin,0,1.5
4,1,187888,hl967ekh,4,6.948973,coronavirus origin,2,3
...,...,...,...,...,...,...,...,...
426,1,167403,bim76jna,987,4.339438,coronavirus origin,0,2
427,1,57151,5i2dxg8z,988,4.335605,coronavirus origin,1,4.5
428,1,129187,zi0lc3lp,991,4.335605,coronavirus origin,2,4.5
429,1,148553,bgh729s5,992,4.335605,coronavirus origin,2,3


Résultat attendu :     


```

qid	docid	docno	rank	score	query	label	iteration
0	1	122804	75773gwg	0	6.948973	coronavirus origin	2.0	5
1	1	122805	kn2z7lho	1	6.948973	coronavirus origin	2.0	3
2	1	122806	4fb291hq	2	6.948973	coronavirus origin	1.0	3
3	1	135326	ne5r4d4b	3	6.948973	coronavirus origin	0.0	1.5
4	1	187888	hl967ekh	4	6.948973	coronavirus origin	2.0	3
...	...	...	...	...	...	...	...	...
995	1	115790	d8gl78lg	995	4.331748	coronavirus origin	0.0	0
996	1	130709	6oiaf2cc	996	4.331748	coronavirus origin	0.0	0
997	1	188659	vagd9i6q	997	4.331748	coronavirus origin	0.0	0
998	1	155287	cetdqgff	998	4.331231	coronavirus origin	0.0	0.5
999	1	80393	ocwzl41r	999	4.329461	coronavirus origin	0.0	0
1000 rows × 8 columns
```

Il existe cependant une fonction qui permet de calculer l'efficacité de ces ordonnancements au travers des métriques d'évaluation (map, précision, rappel, ndcg, ...)

In [ ]:
pt.Experiment(
    [tfidf],
    dataset.get_topics(variant='title'),
    dataset.get_qrels(),
    eval_metrics=["map", "ndcg"])

,name,map,ndcg
0,BR(TF_IDF),0.212085,0.412176


**Exercice 5**

Réaliser une expérience comparant l'expansion de requêtes avec le modèle Bo1 et basé sur la KL-divergence; L'expérience est réalisée sur TREC CORD19 avec le modèle de référence BM25. Vous devrez construire des pipelines appropriées (plus de détails sur [l'expansion](https://pyterrier.readthedocs.io/en/latest/rewrite.html) et les [expérimentations](https://pyterrier.readthedocs.io/en/latest/experiments.html)).

Quelles approches entraînent des augmentations significatives de NDCG et MAP ou autres métriques ?

In [ ]:
pt.Experiment(
    [tfidf],
    dataset.get_topics(variant='title'),
    dataset.get_qrels(),
    eval_metrics=["map", "ndcg"])

,name,map,ndcg
0,BR(TF_IDF),0.212085,0.412176


Résultat attendu :    


```
	name	map	ndcg	ndcg_cut_10
0	+Bo1	0.216699	0.427228	0.593839
1	+RM3	0.219894	0.430525	0.597562
```

